In [1]:
# Load libraries

import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from lightgbm import LGBMRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error
from scipy.stats import skew

In [2]:
%%time
# Read datasets
data = pd.read_csv('./train.csv')
# test = pd.read_csv('./test.csv')

CPU times: user 1min 2s, sys: 3.69 s, total: 1min 6s
Wall time: 1min 6s


In [3]:
'''
log_data = np.log1p(data[features])
test_data = np.log1p(test[features])
log_target = np.log1p(data['target'])
'''

"\nlog_data = np.log1p(data[features])\ntest_data = np.log1p(test[features])\nlog_target = np.log1p(data['target'])\n"

In [4]:
'''
log_target = np.log1p(data['target'])
print(np.min(log_target))
print(np.max(log_target))
'''

"\nlog_target = np.log1p(data['target'])\nprint(np.min(log_target))\nprint(np.max(log_target))\n"

In [5]:
'''
data_min = 30.0
data_max = -1.0
test_min = 30.0
test_max = -1.0
for column in features:
    data_min = min(data_min,np.min(log_data.loc[log_data[column] != 0,column]))
    data_max = max(data_max,np.max(log_data.loc[log_data[column] != 0,column]))
    test_min = min(test_min,np.min(test_data.loc[test_data[column] != 0,column]))
    test_max = max(test_max,np.max(test_data.loc[test_data[column] != 0,column]))
print(data_min)
print(data_max)
print(test_min)
print(test_max)
'''

'\ndata_min = 30.0\ndata_max = -1.0\ntest_min = 30.0\ntest_max = -1.0\nfor column in features:\n    data_min = min(data_min,np.min(log_data.loc[log_data[column] != 0,column]))\n    data_max = max(data_max,np.max(log_data.loc[log_data[column] != 0,column]))\n    test_min = min(test_min,np.min(test_data.loc[test_data[column] != 0,column]))\n    test_max = max(test_max,np.max(test_data.loc[test_data[column] != 0,column]))\nprint(data_min)\nprint(data_max)\nprint(test_min)\nprint(test_max)\n'

In [46]:
%%time
features = [f for f in data.columns if f not in ['target', 'ID']]

# This function generates histogram of each row in datasets
def to_hist_func(row):
    count = row[row != 0].shape[0]
    hist = []
    # Add fine histogram
    for x in np.arange(8,17,0.2):
        hist.append(row[(row < x+1) & (row >= x)].shape[0])
    # Add coarse histogram
    for x in np.arange(8,17,1):
        hist.append(row[(row < x+2) & (row >= x)].shape[0])
    hist.append(row[(row < 23) & (row >= 20)].shape[0])
    hist.append(count) # Add number of nonzero values
    hist.append(skew(row)) # Add skewness
    return hist

# Generate histograms for train data
hist_data = np.apply_along_axis(
    func1d=to_hist_func, 
    axis=1, 
    arr=(np.log1p(data[features])).astype(float)) 



CPU times: user 22.2 s, sys: 318 ms, total: 22.5 s
Wall time: 22.7 s


In [24]:
# column_names = ['hist'+str(f) for f in np.arange(len(hist_data[0]))]
# hist_data_df = pd.DataFrame(hist_data,columns=column_names)
# hist_data[hist_data > 0]
# data_new = pd.concat([data,hist_data_df],axis = 1)
# data_new = data_new.drop(['ID','target'],axis=1)

In [48]:
%%time
# Generate histograms for test data
hist_test = np.apply_along_axis(
    func1d=to_hist_func, 
    axis=1, 
    arr=(np.log1p(test[features])).astype(float))

CPU times: user 5min 59s, sys: 4.79 s, total: 6min 4s
Wall time: 6min 7s


In [9]:

# hist_test_df = pd.DataFrame(hist_test,columns=column_names)
# hist_data[hist_data > 0]
# test_new = pd.concat([test,hist_test_df],axis = 1)
# test_new = test_new.drop(['ID','target'],axis=1)

NameError: name 'hist_test' is not defined

In [50]:
folds = KFold(n_splits=5, shuffle=True, random_state=1)
oof_preds = np.zeros(data.shape[0])
sub_preds = np.zeros(test.shape[0])

for n_fold, (trn_, val_) in enumerate(folds.split(data)):
    reg = ExtraTreesRegressor(
        n_estimators=1500, 
        max_features=.8,                       
        max_depth=12, 
        min_samples_leaf=5, 
        random_state=3, 
        n_jobs=-1
    )
    # Fit Extra Trees
    reg.fit(hist_data[trn_], np.log1p(data['target'].iloc[trn_]))
    # Get OOF predictions
    oof_preds[val_] = reg.predict(hist_data[val_])
    # Update TEST predictions
    sub_preds += reg.predict(hist_test) / folds.n_splits
    # Display fold's score
    print('Fold %d scores : TRN %.4f TST %.4f'
          % (n_fold + 1,
             mean_squared_error(np.log1p(data['target'].iloc[trn_]),
                                reg.predict(hist_data[trn_])) ** .5,
             mean_squared_error(np.log1p(data['target'].iloc[val_]),
                                reg.predict(hist_data[val_])) ** .5))
          
print('Full OOF score : %.4f' % (mean_squared_error(np.log1p(data['target']), oof_preds) ** .5))

Fold 1 scores : TRN 0.9902 TST 1.3218
Fold 2 scores : TRN 0.9724 TST 1.3689
Fold 3 scores : TRN 0.9844 TST 1.3214
Fold 4 scores : TRN 0.9779 TST 1.3389
Fold 5 scores : TRN 0.9900 TST 1.2774
Full OOF score : 1.3260


In [51]:
#customerid = data['ID']
#result = pd.Series(gbm.predict(test_x),name='target')
#result = pd.Series(oof_preds,name='target')
#result = pd.concat([customerid,result],axis=1)
#min_value = train_y.min()
#result.loc[result['target'] < min_value,'target'] = min_value
#result.to_csv('Santander_train_2.csv',index=False)

# Generate submission file
test['target'] = np.expm1(sub_preds)
test[['ID', 'target']].to_csv('histogram_predictions.csv', index=False)

In [ ]:
# result['target_ob'] = np.log1p(data['target'])
# result['difference'] = (result['target']-result['target_ob'])**2
# result_2 = pd.read_csv('../Results/Santander_train_1.csv')
# result['target_2'] = result_2['target']

In [ ]:
# pd.set_option('display.max_rows',5000)
# result[(result['target'] < 15) & (result['target'] > 14)]
# result_3 = []
# for ind, row in result_2.iterrows():
    # result_3.append((result.iloc[ind]['target']+result_2.iloc[ind]['target'])/2)

In [ ]:
# print('Full Out-Of-Fold score : %9.6f' 
      # % (mean_squared_error(np.log1p(data['target']), result_3) ** .5))